# Generate instance of Knapsack

In [47]:
#Suppose we have a instance of 5 items


import random
from pyscipopt import Model, quicksum

# -----------------------------
# 1. Generate Random Instance
# -----------------------------
def generate_knapsack_instance(num_items=5, max_weight=20, max_value=50, seed=40):
    random.seed(seed)
    weights = [random.randint(1, max_weight) for _ in range(num_items)]
    values = [random.randint(1, max_value) for _ in range(num_items)]
    capacity = int(sum(weights) * 0.6)  # set capacity to 60% of total weight
    return weights, values, capacity

# -----------------------------

weights, values, capacity = generate_knapsack_instance()

# Solve instance using IP

In [48]:
# 2. Build and Solve in SCIP
# -----------------------------
def solve_knapsack(weights, values, capacity):
    n = len(weights)
    model = Model("0-1_Knapsack")

    # Decision variables: x[i] = 1 if item i is chosen
    x = {i: model.addVar(vtype="B", name=f"x_{i}") for i in range(n)}

    # Objective: maximize value
    model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

    # Weight constraint
    model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

    # Solve
    model.optimize()

    # Extract solution
    solution = {i: model.getVal(x[i]) for i in range(n)}
    total_value = sum(values[i] * solution[i] for i in range(n))
    total_weight = sum(weights[i] * solution[i] for i in range(n))
    
    return solution, total_value, total_weight


# Solve instance using LP relaxation

In [49]:
#Let's solve the LP relaxation

def solve_knapsack_lp_relaxation(weights, values, capacity):
    n = len(weights)
    model = Model("Knapsack_LP_Relax")

    # LP variables: 0 <= x[i] <= 1 (continuous)
    x = {i: model.addVar(lb=0.0, ub=1.0, vtype="C", name=f"x_{i}") for i in range(n)}

    # Maximize total value
    model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

    # Capacity constraint
    model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

    # Solve
    model.optimize()

    # Extract (possibly fractional) solution
    sol = {i: model.getVal(x[i]) for i in range(n)}
    total_value = sum(values[i] * sol[i] for i in range(n))
    total_weight = sum(weights[i] * sol[i] for i in range(n))
    return sol, total_value, total_weight


In [50]:
print("Item Weights:", weights)
print("Item Values :", values)
print("Knapsack Capacity:", capacity)

solution, total_value, total_weight = solve_knapsack(weights, values, capacity)

print("\nChosen Fractions (LP relaxation):")
for i in range(len(weights)):
    print(f"Item {i}: x={solution[i]:.4f}, weight={weights[i]}, value={values[i]}")
print(f"\nTotal Weight = {total_weight:.4f}")
print(f"Total Value  = {total_value:.4f}")


# -----------------------------
# Run everything
# -----------------------------

solution, total_value, total_weight = solve_knapsack_lp_relaxation(weights, values, capacity)

print("\nChosen Fractions (LP relaxation):")
for i in range(len(weights)):
    print(f"Item {i}: x={solution[i]:.4f}, weight={weights[i]}, value={values[i]}")
print(f"\nTotal Weight = {total_weight:.4f}")
print(f"Total Value  = {total_value:.4f}")


Item Weights: [15, 19, 17, 2, 8]
Item Values : [19, 43, 41, 43, 14]
Knapsack Capacity: 36
feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:

Chosen Fractions (LP relaxation):
Item 0: x=1.0000, weight=15, value=19
Item 1: x=1.0000, weight=19, value=43
Item 2: x=-0.0000, weight=17, value=41
Item 3: x=1.0000, weight=2, value=43
Item 4: x=-0.0000, weight=8, value=14

Total Weight = 36.0000
Total Value  = 105.0000

Chosen Fractions (LP relaxation):
Item 0: x=0.0000, weight=15, value=19
Item 1: x=0.8947, weight=19, value=43
Item 2: x=1.0000, weight=17, value=41
Item 3: x=1.0000, weight=2, value=43
Item 4: x=0.0000, weight=8, value=14

Total Weight = 36.0000
Total Value  = 122.4737
   (0.0s) running MILP presolver
   (0.3s) MILP presolver found nothing
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 0 del conss, 0 add conss

# Let's identify the optimal solution through enumeration

In [51]:
def solve_model_w_constraints(weights, values, capacity, constraints):
    
    n = len(weights)
    model = Model("Knapsack_LP_Relax")

    # LP variables: 0 <= x[i] <= 1 (continuous)
    x = {i: model.addVar(lb=0.0, ub=1.0, vtype="C", name=f"x_{i}") for i in range(n)}

    # Maximize total value
    model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

    # Capacity constraint
    model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)
    
    #Additional constraints:
    for var, constraint_val in constraints.items():
        model.addCons(x[var] == constraint_val)

    # Solve
    model.optimize()
    
    status = model.getStatus()
    
    if status == "optimal":
        # Extract (possibly fractional) solution
        sol = {i: model.getVal(x[i]) for i in range(n)}
        total_value = sum(values[i] * sol[i] for i in range(n))
        total_weight = sum(weights[i] * sol[i] for i in range(n))
        return sol, total_value, total_weight
    else:
        return None, None, None


In [52]:
def find_optimal_value(weights, values, capacity):
    V = []
    A = []
    A.append({}) #add root node
    while len(A) > 0:
        node = A.pop() #node is just the constraints of that node
        sol, total_value, total_weight = solve_model_w_constraints(weights, values, capacity, node) #solve LP relaxation of node
        if sol == None: #pop next node if this one is infeasible
            continue
        elif all(float(sol_val).is_integer() for sol_val in sol.values()): #if integer feasible
                V.append(total_value)
        else: #if fractional relaxation
            
            #identify which var to branch on
            fractional_keys = [key for key, v in sol.items() if abs(v - round(v)) > 1e-9]
            branch_var = fractional_keys[0]
            
            #add two new nodes to A
            node1 = node.copy()
            node2 = node.copy()
            node1[branch_var] = 1
            node2[branch_var] = 0
            A.append(node1)
            A.append(node2)
    
    print(f"{max(V)} IS THE OPTIMAL VALUE ")
    return max(V)
            
            
            
            
            
        
    